In [ ]:
using LibGit2
using JSON

In [ ]:
function GetMostRecentJsonFile(folderContainingRegistryJson::String, objectType::String)
    for filename in readdir(folderContainingRegistryJson, sort=true)
        if startswith(filename, objectType)
            return joinpath(folderContainingRegistryJson, filename)
        end
    end
end

function GetJson(reposJsonFilepath)
    io = open(reposJsonFilepath, "r")
    reposJson = String(read(io))
    close(io)
    json = JSON.parse(reposJson)
    return json 
end

function GetReposJson(objectType::String)
    reposJsonFilepath = GetMostRecentJsonFile("TfRegistry2JSON", objectType)
    return GetJson(reposJsonFilepath)
end

In [ ]:
function InitializeVariables(reposJson)
   return 0, length(reposJson), Vector{Any}(), Vector{String}() 
end

In [ ]:
function PrepareSubfolderWrite(objectType::String)
    subfolder = "TfCloneRepos_" * objectType
    if ! isdir(subfolder)
        mkdir(subfolder)
    end
    return joinpath(pwd(), subfolder)
end

In [ ]:
function PrepareRepoSubfolderWrite(subfolderPath, username, reponame)
    userFolder = joinpath(subfolderPath, username)
    if ! isdir(userFolder)
        mkdir(userFolder)
    end
    repoFolder = joinpath(userFolder, reponame)
    if ! isdir(repoFolder)
        mkdir(repoFolder)
    end
    versionFolder = joinpath(repoFolder, "main")
    if ! isdir(versionFolder)
        mkdir(versionFolder)
        return versionFolder
    else
        # We also need to remove folders that are empty, to save errors later
        if (length(readdir(versionFolder)) == 0)
            rm(versionFolder)
        end
        return ""
    end
end

In [ ]:
function CloneRepo(githubUrl, mainFolder, errors)
    print("Cloning : " * githubUrl)
    try
        repo = LibGit2.clone(githubUrl, mainFolder, callbacks = LibGit2.Callbacks(:credentials => (C_NULL, 0)))
        println(", done.")
        return repo
    catch err
        push!(errors, githubUrl * " - " * string(err))
        println("\n\tError. " * githubUrl * " - " * string(err))
    end
end

In [ ]:
function CloneAllRepos(objectType::String)
    
    reposJson = GetReposJson(objectType)
    
    subfolderPath = PrepareSubfolderWrite(objectType)
    
    count, total = 0, length(reposJson)
    errors = Vector{Any}()
    
    for repoKey in collect(keys(reposJson))
        
        count += 1
        print("$count out of $total : ")
        
        username, reponame = split(repoKey, "/")
        mainFolder = PrepareRepoSubfolderWrite(subfolderPath, username, reponame)
        
        # We already have a version of this repo, we're supposed to add new repos
        if mainFolder == ""
            println()
            continue
        end
        
        # Clone the new repo
        githubUrl = "https://github.com/" * username * "/" * reponame
        repo = CloneRepo(githubUrl, mainFolder, errors)
    end
    
    println("num errors: " * string(length(errors)))
    return errors
end

In [ ]:
@time errorProviders = CloneAllRepos("providers")

In [ ]:
@time errorModules = CloneAllRepos("modules")